In [1]:
import numpy as np
import pandas as pd
import nltk
# nltk.download("punkt")
import re
import networkx as nx

from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

### Lecture du fichier

In [2]:
f = open("resultat/cgu_figaro_full.txt", "r")
s = f.read()
f.close()

### Identification des phrases dans un corpus

In [3]:
sentences = sent_tokenize(s)

#sentences = [y for x in sentences for y in x] # flatten list

In [4]:
sentences[:5]

["Retour au site \n\n\nSkip to main content\n\n\nMain menuCGUPolitique de confidentialitéInfos cookiesMon FigaroCharte de modérationMentions légalesFAQ\n\n\n\n\n\n\n\n\n\n\nConditions Générales d'Utilisation\n\n\xa0\nSociété du Figaro, SAS au capital de 16.860.475 €, dont le siège est 14, bd Haussmann, 75009 PARIS, immatriculée au Registre du Commerce et des Sociétés de Paris sous le numéro 542 077 755 (ci-après « Le Figaro »), édite un site d'informations générales www.lefigaro.fr, ci-après le « Site » permettant un accès à différentes informations, ainsi qu'à différents services et produits complémentaires, détaillés ci-après.",
 'Les présentes conditions générales d\'utilisation, ("CGU"), soumises au droit français, ont vocation à régir l\'utilisation du Site ainsi que l\'utilisation des différents services payants ou gratuits (ci-après le(s) «\xa0Service(s)\xa0»).',
 "La navigation dans le Site et/ou l'inscription à un Service emporte de plein droit acceptation expresse et sans rés

### Lecture du dictionnaire de donnée

In [6]:
word_embeddings = {}
# f = open('ressources/multilingual_embeddings.fr', encoding='utf8')
f = open('../../files/vectors_2.txt', encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()
len(word_embeddings)

41153

In [7]:
len(word_embeddings[list(word_embeddings)[0]])

300

### Netoyage des phrases

In [8]:
clean_sentences = []
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Zàéêèç]" , " ")
clean_sentences = [sentence.lower() for sentence in clean_sentences]
clean_sentences = [sentence for sentence in clean_sentences if sentence != "" and sentence != " "]
clean_sentences[:5]

['retour au site    skip to main content   main menucgupolitique de confidentialitéinfos cookiesmon figarocharte de modérationmentions légalesfaq           conditions générales d utilisation    société du figaro  sas au capital de               dont le siège est     bd haussmann        paris  immatriculée au registre du commerce et des sociétés de paris sous le numéro              ci après   le figaro     édite un site d informations générales www lefigaro fr  ci après le   site   permettant un accès à différentes informations  ainsi qu à différents services et produits complémentaires  détaillés ci après ',
 'les présentes conditions générales d utilisation    cgu    soumises au droit français  ont vocation à régir l utilisation du site ainsi que l utilisation des différents services payants ou gratuits  ci après le s    service s     ',
 'la navigation dans le site et ou l inscription à un service emporte de plein droit acceptation expresse et sans réserve des cgu par l utilisateur d

### Suppression des stop_words

In [9]:
stop_words = stopwords.words('french')
type(stop_words)
stop_words[:5]

['au', 'aux', 'avec', 'ce', 'ces']

In [10]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
clean_sentences[:5]

['retour site skip to main content main menucgupolitique confidentialitéinfos cookiesmon figarocharte modérationmentions légalesfaq conditions générales utilisation société figaro sas capital dont siège bd haussmann paris immatriculée registre commerce sociétés paris sous numéro ci après figaro édite site informations générales www lefigaro fr ci après site permettant accès différentes informations ainsi différents services produits complémentaires détaillés ci après',
 'présentes conditions générales utilisation cgu soumises droit français vocation régir utilisation site ainsi utilisation différents services payants gratuits ci après service',
 'navigation site inscription service emporte plein droit acceptation expresse sans réserve cgu utilisateur site',
 'cas non respect termes présentes cgu figaro réserve droit prendre toute mesure nature préserver intérêts notamment assurer exécution',
 'figaro pourra notamment cette fin assigner utilisateur site responsabilité civile pénale']

### Transformation des phrases en vecteur à partir du dictionnaire

In [12]:
sentence_vectors = []
for i in clean_sentences:
    # print(i)
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((300,))) for w in i.split()])/(len(i.split()) + 0.001)
    else:
        v =np.zeros((300,))
    sentence_vectors.append(v)
sentence_vectors[:1]

[array([ 0.06740985, -0.08534208,  0.04874635, -0.04992397, -0.19614087,
        -0.07718496, -0.15040013, -0.06708689, -0.01779479,  0.03138459,
         0.10189346,  0.0775622 , -0.12328808, -0.08983755,  0.07476393,
        -0.00876111,  0.07035271, -0.00768427,  0.06331701, -0.10446227,
        -0.00617337, -0.03733839,  0.06535843, -0.01040354, -0.04864981,
         0.05831401,  0.04467544, -0.00499011, -0.03051306,  0.03104779,
         0.02308777, -0.05186191, -0.06903579,  0.10598169, -0.06838745,
        -0.04241943, -0.02684551, -0.07316549, -0.01642373, -0.00772214,
        -0.05626671, -0.10866827,  0.0032481 ,  0.05649099,  0.03221354,
        -0.01059152,  0.06855975,  0.01757622,  0.02335791,  0.09495484,
         0.06567203,  0.01227649, -0.07783483,  0.01826552, -0.05636352,
         0.00630463,  0.01391611, -0.20803049, -0.03922084,  0.10925261,
        -0.10078011, -0.05820489, -0.01790139,  0.05931975, -0.03773485,
        -0.06219903,  0.00696886, -0.02204481,  0.0

### Calcul du scores similarité entre les vecteurs

In [13]:
sim_mat = np.zeros([len(sentences), len(sentences)])
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,300), sentence_vectors[j].reshape(1, 300))[0,0]
sim_mat[1, 2]

0.7870632010066971

### Utilisation de l'alorithme pagerank qui permet de classifier les scores.
Conversion de la matrice de similarité en graph.
Les noeuds de ce graphique représenteront les phrases et les edges représenteront les scores de similitude entre les phrases.
Nous classons ensuite les phrases à l'aide de l'algorithme pagerank qui permet de classifier les phrases en fonction de score.

In [14]:
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)
scores

{0: 0.004309292972256052,
 1: 0.00433934098269049,
 2: 0.004340025762612502,
 3: 0.003782715498267305,
 4: 0.0040578195008554524,
 5: 0.0025273763353161277,
 6: 0.002544543239902796,
 7: 0.004535331812422545,
 8: 0.003995515132773333,
 9: 0.004297089154467119,
 10: 0.0040749176670772606,
 11: 0.004259444312822262,
 12: 0.004800908004460315,
 13: 0.004507485844615996,
 14: 0.0041342244655530585,
 15: 0.004058158339383331,
 16: 0.004327011365250587,
 17: 0.003572871918303658,
 18: 0.0036786663990393863,
 19: 0.0038548941451973903,
 20: 0.004388638310499372,
 21: 0.004156931904815838,
 22: 0.003968569566592023,
 23: 0.002443573590040608,
 24: 0.003776466351098186,
 25: 0.001630526813585375,
 26: 0.0043985273587984,
 27: 0.0033658024354789166,
 28: 0.0009602637239761472,
 29: 0.004120936432683819,
 30: 0.0032777568059143215,
 31: 0.004611722417642041,
 32: 0.004046849175145952,
 33: 0.0030482496129790497,
 34: 0.00405801076087496,
 35: 0.004243590095102914,
 36: 0.004595213204087562,
 37: 

### Trie et récupération des phrases correspondant au scores calculé précédemment et ajouts des phrases avec le meilleur score dans 'resultat'

In [15]:
resultat = ""
ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
for i in range(10):
    resultat += ranked_sentences[i][1] + "\n\n"

In [16]:
print(resultat)

Notre Site ou l’Application contient des cookies des liens de partage vers Facebook, Twitter et autre réseau social similaire, qui vous permettent de partager des contenus de notre Site ou de notre Application avec d’autres personnes ou de faire connaître à ces autres personnes votre consultation ou votre opinion concernant un contenu du Site/Application ou encore de vous connecter pour accéder à l'ensemble des services de la Société du Figaro en utilisant vos données d’identification sur ces réseaux sociaux.

Les cookies et identifiants publicitaires permettent aux sociétés du Groupe Figaro ainsi qu’à nos partenaires commerciaux d’afficher sur notre Site/Application ou sur les sites de tiers des publicités ou des liens sponsorisés adaptées à vos centres d’intérêts selon votre profil d’utilisateur, voire, le cas échéant, en fonction des informations concernant votre navigation sur notre Site/Application, et des données concernant votre position géographique (longitude et latitude) tran

In [17]:
# f = open("resultat/resultat_summarization_fr_v2.txt", "w")
# f.write(resultat)
# f.close()

In [18]:
print(ranked_sentences)

[(0.004894784969538741, "Notre Site ou l’Application contient des cookies des liens de partage vers Facebook, Twitter et autre réseau social similaire, qui vous permettent de partager des contenus de notre Site ou de notre Application avec d’autres personnes ou de faire connaître à ces autres personnes votre consultation ou votre opinion concernant un contenu du Site/Application ou encore de vous connecter pour accéder à l'ensemble des services de la Société du Figaro en utilisant vos données d’identification sur ces réseaux sociaux."), (0.004857749362442655, 'Les cookies et identifiants publicitaires permettent aux sociétés du Groupe Figaro ainsi qu’à nos partenaires commerciaux d’afficher sur notre Site/Application ou sur les sites de tiers des publicités ou des liens sponsorisés adaptées à vos centres d’intérêts selon\xa0votre profil d’utilisateur, voire, le cas échéant, en fonction des informations concernant votre navigation sur notre Site/Application, et des données concernant vo

In [20]:
with open('../../files/vectors_2.txt', encoding='utf8') as myfile:
    [print(next(myfile)) for x in range(40)]
    print("\n\n\n")

ï 0.352455 0.163925 -0.162153 0.495864 -0.625714 0.244730 0.111394 0.273667 -0.956016 0.654493 -0.043997 -0.178529 -0.456594 0.700850 -0.886346 -0.132669 -0.338248 -0.262157 -0.261616 0.445262 -0.186543 -0.419467 0.672910 -0.518146 -0.684908 -0.157855 -0.130967 -0.754243 -0.774289 0.617461 0.066966 0.405432 -0.469119 -0.886351 -0.894592 -1.018018 -0.326838 0.212480 0.168514 -0.089929 -0.143742 0.003387 -0.403038 0.813745 0.467102 -0.927733 -0.237010 0.501869 0.119843 0.294325 -0.097892 -0.104813 0.367958 -0.190739 0.794975 0.374013 0.399962 -0.346863 0.779067 0.691698 0.431872 0.417248 -0.258779 0.318999 -0.162764 0.486080 -0.244018 -0.259531 0.105026 -0.893309 0.111514 0.306578 -0.124978 -0.628067 -0.067492 1.044546 0.013348 0.096935 -0.473368 -0.396542 -0.154648 -0.268957 0.139823 0.150760 -0.840313 -0.655187 -0.732035 0.542768 0.595926 -0.324315 0.030582 -0.874914 -0.199525 0.580986 0.354904 -0.080118 0.054057 0.194345 -0.475827 -1.713534 0.144545 -0.220685 -0.299538 -0.256061 0.404